# Test conditional constraints

## Example
The constraint to check is :    

    if the observationType is an animal, the scientificName has to be not null
    
A DataFrame example is used to show the validation of the constraint

In [1]:
import pandas as pd

df_ko = pd.DataFrame({'observationType': ['animal', 'tree', 'animal'], 
                      'scientificName': ['Vulpes vulpes', 'null', 'null']})

df_ok = pd.DataFrame({'observationType': ['animal', 'tree', 'animal'], 
                      'scientificName': ['Vulpes vulpes', 'null', 'Vulpes cana']})

## option 1 : ifField

`ifField` defines a boolean Field used as a filter.    
`thenField` defines a constraints applicable to the filtered Field (True values of the filter)    
`elseField` defines a constraints applicable to the filtered Field (False values of the filter)    


In [2]:
# example of schema

{"ifField": {
    "observationType": { "const": "animal" }}, # defines a boolean Field
 "thenField": {
    "scientificName": { "not": {"const": "null"}}} #constraint for the 'scientific_name' Field
}


{'ifField': {'observationType': {'const': 'animal'}},
 'thenField': {'scientificName': {'not': {'const': 'null'}}}}

In [3]:
# pandas equivalence

if_field_ok = df_ok['observationType'] == 'animal'
then_field_ok = (df_ok.loc[if_field_ok]['scientificName'] != 'null').all()

if_field_ko = df_ko['observationType'] == 'animal'
then_field_ko = (df_ko.loc[if_field_ko]['scientificName'] != 'null').all()

print(then_field_ok, then_field_ko)

True False


## option 2 : allOfField-anyOfField

`allOfField` is an AND between boolean Fields.     
`anyOfField`  is an OR between boolean Fields 



In [6]:
# example of schema

{"anyOfField": { 
    "observationType": { "not": { "const": "animal" }}, # defines a boolean Field
    "scientificName": { "not": {"const": "null"}}} # defines a boolean Field
}
# 'if A then B' is equivalent as 'B or Not A'

{'anyOfField': {'observationType': {'not': {'const': 'animal'}},
  'scientificName': {'not': {'const': 'null'}}}}

In [5]:
# pandas equivalence

bool_field1_ok = df_ok['observationType'] != 'animal'
bool_field2_ok = df_ok['scientificName'] != 'null'
valid_ok = (bool_field1_ok | bool_field2_ok).all()

bool_field1_ko = df_ko['observationType'] != 'animal'
bool_field2_ko = df_ko['scientificName'] != 'null'
valid_ko = (bool_field1_ko | bool_field2_ko).all()

print(valid_ok, valid_ko)

True False
